# Machine Translation From Arabic to English Using Spark NLP

## Colab Setup

In [ ]:
! pip install -q pyspark==3.3.0 spark-nlp==4.2.8
! pip install --upgrade -q spark-nlp-display

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 453.8/453.8 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.6/95.6 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.4 MB/s eta 0:00:00


## Start the Spark session

Import dependencies and start Spark session.

In [ ]:
import json
import pandas as pd
import numpy as np

import sparknlp
import pyspark.sql.functions as F

from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from sparknlp.annotator import *
from sparknlp.base import *
from sparknlp.pretrained import PretrainedPipeline
from pyspark.sql.types import StringType, IntegerType

In [ ]:
spark = sparknlp.start()

print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

spark

Spark NLP version 4.2.8
Apache Spark version: 3.3.0


## Demo text to be translated to english

In [ ]:
text = """اليوم سنناقش الطقس في دبي"""
#testing w ar

Demo for testing arabic news article

In [ ]:
!pip install -q newspaper3k
import newspaper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 31.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done


In [ ]:
# Extract the article text from the URL
article = newspaper.Article("https://www.aljazeera.net/politics/2024/6/9/%D9%88%D8%AB%D8%A7%D8%A6%D9%82-%D9%85%D8%B3%D8%B1%D8%A8%D8%A9-%D8%AA%D9%83%D8%B4%D9%81-%D8%B7%D9%84%D8%A8-%D8%A3%D9%85%D9%8A%D8%B1%D9%83%D8%A7-%D9%85%D9%86-%D8%AD%D9%84%D9%81%D8%A7%D8%A6%D9%87%D8%A7")
article.download()
article.parse()
text = article.text

In [ ]:
print(text)

كشفت وثائق حصلت عليها صحيفة "تايمز أوف إسرائيل" أن الولايات المتحدة طلبت من حلفائها العرب عدم تحديد موعد نهائي لتنفيذ حل الدولتين بعد أن تضع الحرب في قطاع غزة أوزارها.

وورد في تلك الوثائق أن إدارة الرئيس الأميركي جو بايدن تسعى لمنع حلفائها العرب من طرح رؤية بعيدة المدى لتسوية سلمية بين الإسرائيليين والفلسطينيين بعد انتهاء الحرب في غزة، والاستعاضة عنها بإطار عام أضيق نطاقا تؤكد الصحيفة أن حكومة رئيس الوزراء بنيامين نتنياهو سترفضه.

وأشارت في تقرير لمدير مكتبها في الولايات المتحدة، جاكوب ماجد، إلى أن واشنطن ظلت، منذ بداية العام، تقود مجموعة اتصال من كبار الوزراء من السعودية والإمارات العربية المتحدة وقطر ومصر والأردن والسلطة الفلسطينية بهدف الدفع بخطة لإدارة قطاع غزة بعد الحرب.

وفي أبريل/نيسان، فرغ الوزراء العرب -في اجتماع للتنسيق كمجموعة بشكل مستقل عن الولايات المتحدة- من صياغة رؤيتهم لما بعد الحرب، والتي تضمنت اعترافا دوليا فوريا بالدولة الفلسطينية، وإنشاء قوة لحفظ السلام في الضفة الغربية والقدس الشرقية وإطلاق محادثات سلام بين إسرائيل والسلطة الفلسطينية على أن تستكمل خلال عامين، بما 

=## Spark Pipeline

In [ ]:
documentAssembler = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")

## DL model can detect start and end of setence ==> more accurate
sentencerDL = SentenceDetectorDLModel()\
  .pretrained("sentence_detector_dl", "xx")\
  .setInputCols(["document"])\
  .setOutputCol("sentences")

marian = MarianTransformer.pretrained("opus_mt_ar_en", "xx")\
  .setInputCols(["sentences"])\
  .setOutputCol("translation")

nlp_pipeline = Pipeline(
    stages=[
        documentAssembler,
        sentencerDL,
        marian
        ])

sentence_detector_dl download started this may take some time.
Approximate size to download 514.9 KB
[OK!]
opus_mt_ar_en download started this may take some time.
Approximate size to download 390.7 MB
[OK!]


## Run the pipeline

In [ ]:
empty_df = spark.createDataFrame([[""]]).toDF('text')
pipeline_model = nlp_pipeline.fit(empty_df)
lmodel = LightPipeline(pipeline_model)
res = lmodel.fullAnnotate(text)


Before _validateStagesInputCols


## Results for Terminal

In [ ]:
print ('Original:', text, '\n\n')

print ('Translated:\n')
for sentence in res[0]['translation']:
  print (sentence.result)

Original: كشفت وثائق حصلت عليها صحيفة "تايمز أوف إسرائيل" أن الولايات المتحدة طلبت من حلفائها العرب عدم تحديد موعد نهائي لتنفيذ حل الدولتين بعد أن تضع الحرب في قطاع غزة أوزارها.

وورد في تلك الوثائق أن إدارة الرئيس الأميركي جو بايدن تسعى لمنع حلفائها العرب من طرح رؤية بعيدة المدى لتسوية سلمية بين الإسرائيليين والفلسطينيين بعد انتهاء الحرب في غزة، والاستعاضة عنها بإطار عام أضيق نطاقا تؤكد الصحيفة أن حكومة رئيس الوزراء بنيامين نتنياهو سترفضه.

وأشارت في تقرير لمدير مكتبها في الولايات المتحدة، جاكوب ماجد، إلى أن واشنطن ظلت، منذ بداية العام، تقود مجموعة اتصال من كبار الوزراء من السعودية والإمارات العربية المتحدة وقطر ومصر والأردن والسلطة الفلسطينية بهدف الدفع بخطة لإدارة قطاع غزة بعد الحرب.

وفي أبريل/نيسان، فرغ الوزراء العرب -في اجتماع للتنسيق كمجموعة بشكل مستقل عن الولايات المتحدة- من صياغة رؤيتهم لما بعد الحرب، والتي تضمنت اعترافا دوليا فوريا بالدولة الفلسطينية، وإنشاء قوة لحفظ السلام في الضفة الغربية والقدس الشرقية وإطلاق محادثات سلام بين إسرائيل والسلطة الفلسطينية على أن تستكمل خلال ع

In [ ]:
sentence = ""
for s in res[0]['translation']:
  sentence = sentence + s.result

print(sentence)

Documents obtained by Times of Israel revealed that the United States had asked its Arab allies not to set a deadline for the implementation of the two-State solution after the war in the Gaza Strip had been putIt is stated in those documents that the administration of US President Joe Biden is seeking to prevent its Arab allies from presenting a far-reaching vision for a peaceful settlement between Israelis and Palestinians after theIn a report by her United States Bureau Director, Jacob Majid, she noted that since the beginning of the year Washington had been leading a contact group of senior ministers from Saudi Arabia, the United ArabIn April, Arab ministers at a coordination meeting as a group independent of the United States finalized their vision of the post-war era, which included immediate international recognition of the Palestinian State and the establishment ofAlthough the US administration supports the broader two-State solution that its six Arab partners are trying to mov

#Summarization

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
summary = summarizer(sentence)
if not summary or not summary[0] or 'summary_text' not in summary[0]:
    print( "Failed to summarize the text.", "Failed to summarize the text.")
summarized_text = summary[0]['summary_text']

In [ ]:
print(summarized_text)

Documents obtained by Times of Israel revealed that the United States had asked its Arab allies not to set a deadline for the implementation of the two-State solution after the war in the Gaza Strip. The administration of US President Joe Biden is seeking to prevent Arab allies from presenting a far-reaching vision for a peaceful settlement.


# Deploying the Spark Based Model using Flask and ngrok

In [ ]:
!pip install Flask

In [ ]:
!pip install pyngrok

In [ ]:
!pip install Flask pyngrok
!pip install -q pyspark==3.3.0 spark-nlp==4.2.8
!pip install --upgrade -q spark-nlp-display

In [ ]:
import json
import pandas as pd
import numpy as np
from flask import Flask, request, render_template_string
import sparknlp
from sparknlp.pretrained import PretrainedPipeline
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import SentenceDetectorDLModel, MarianTransformer
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from sparknlp.base import LightPipeline
from transformers import pipeline as transformers_pipeline
from newspaper import Article

In [ ]:
spark = sparknlp.start()

summarizer = transformers_pipeline("summarization", model="facebook/bart-large-cnn")

def get_pipeline():
    documentAssembler = DocumentAssembler().setInputCol("text").setOutputCol("document")
    sentencerDL = SentenceDetectorDLModel().pretrained("sentence_detector_dl", "xx").setInputCols(["document"]).setOutputCol("sentences")
    marian = MarianTransformer.pretrained("opus_mt_ar_en", "xx").setInputCols(["sentences"]).setOutputCol("translation")

    nlp_pipeline = Pipeline(stages=[documentAssembler, sentencerDL, marian])
    empty_df = spark.createDataFrame([[""]]).toDF('text')
    pipeline_model = nlp_pipeline.fit(empty_df)
    lmodel = LightPipeline(pipeline_model)
    return lmodel

pipeline = get_pipeline()

# Function to extract text from a URL using newspaper3k
def extract_text_from_url(url):
    article = Article(url)
    article.download()
    article.parse()
    return article.text
def summarize_text(text):
    summary = summarizer(text)
    if not summary or not summary[0] or 'summary_text' not in summary[0]:
        return "Failed to summarize the text."
    return summary[0]['summary_text']

# Starting Flask app
app = Flask(__name__)

# HTML template as a string
template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Machine Translation</title>
</head>
<body>
    <h1>Machine Translation</h1>
    <form method="POST" action="/translate">
        <label for="text">Text to translate:</label><br>
        <textarea id="text" name="text" rows="4" cols="50"></textarea><br><br>

        <input type="submit" value="Translate">
    </form>
    <h1>URL Translation and Summarization</h1>
    <form method="POST" action="/translate_url">
        <label for="url">URL to translate and summarize:</label><br>
        <input type="text" id="url" name="url" size="50"><br><br>

        <input type="submit" value="Translate and Summarize">
    </form>
    {% if original_text %}
        <h2>Original Text:</h2>
        <p>{{ original_text }}</p>
        <h2>Translated Text:</h2>
        <p>{{ translated_text }}</p>
    {% endif %}
    {% if summarized_text %}
        <h2>Summarized Text:</h2>
        <p>{{ summarized_text }}</p>
    {% endif %}
</body>
</html>
"""

@app.route('/')
def index():
    return render_template_string(template)

@app.route('/translate', methods=['POST'])
def translate():
    text = request.form['text']
    res = pipeline.fullAnnotate(text)
    translated_text = " ".join([sentence.result for sentence in res[0]['translation']])
    return render_template_string(template, original_text=text, translated_text=translated_text)

@app.route('/translate_url', methods=['POST'])
def translate_url():
    url = request.form['url']
    original_text = extract_text_from_url(url)
    res = pipeline.fullAnnotate(original_text)
    translated_text = " ".join([sentence.result for sentence in res[0]['translation']])
    summarized_text = summarize_text(translated_text)
    return render_template_string(template, original_text=original_text, translated_text=translated_text, summarized_text=summarized_text)

# Run Flask app
from pyngrok import ngrok

# Set ngrok token
NGROK_AUTH_TOKEN = "YOUR AUTH TOKEN"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

public_url = ngrok.connect(5000)
print(f'Public URL: {public_url}')
app.run(port=5000)

sentence_detector_dl download started this may take some time.
Approximate size to download 514.9 KB
[OK!]
opus_mt_ar_en download started this may take some time.
Approximate size to download 390.7 MB
[OK!]


Public URL: NgrokTunnel: "https://a3d9-34-91-45-112.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [11/Jun/2024 14:17:13] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Jun/2024 14:17:14] "GET /favicon.ico HTTP/1.1" 404 -


Before _validateStagesInputCols
Before _validateStagesInputCols


INFO:werkzeug:127.0.0.1 - - [11/Jun/2024 14:22:44] "POST /translate_url HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Jun/2024 14:27:38] "POST /translate_url HTTP/1.1" 200 -


Before _validateStagesInputCols


In [ ]:

spark = sparknlp.start()

def get_pipeline():
    documentAssembler = DocumentAssembler().setInputCol("text").setOutputCol("document")
    sentencerDL = SentenceDetectorDLModel().pretrained("sentence_detector_dl", "xx").setInputCols(["document"]).setOutputCol("sentences")
    marian = MarianTransformer.pretrained("opus_mt_ar_en", "xx").setInputCols(["sentences"]).setOutputCol("translation")

    nlp_pipeline = Pipeline(stages=[documentAssembler, sentencerDL, marian])
    empty_df = spark.createDataFrame([[""]]).toDF('text')
    pipeline_model = nlp_pipeline.fit(empty_df)
    lmodel = LightPipeline(pipeline_model)
    return lmodel

# this will load the above pipeline to do MT ar ==> eng
pipeline = get_pipeline()

# Starting Flask app
app = Flask(__name__)

# Colab cannot take HTML files as is so declare as a string and then use
template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Arabic to English</title>
    <style>
        h1{
            font-size: 42pt;
            font-family: Impact, Haettenschweiler, 'Arial Narrow Bold', sans-serif;
            align-items: center;
            color: rgb(33, 21, 6);
            justify-content: center;
        }
        body{
            background-color: rgb(167, 158, 216);
        }
        p{
            font-size: 20pt;
            font-style: italic;
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            align-items: center;
            justify-content: center;
        }
    </style>
</head>
<body>
    <h1>Arabic To English</h1>
    <h2>Machine Translation</h2>
    <p>This is a Spark NLP based model that takes input in <b>Arabic</b> and changes it to <b>English</b></p>
    <form method="POST" action="/translate">
        <label for="text">Text to translate:</label><br>
        <textarea id="text" name="text" rows="4" cols="50"></textarea><br><br>

        <input type="submit" value="Translate">
    </form>
    {% if original_text %}
        <h2>Original Text:</h2>
        <p>{{ original_text }}</p>
        <h2>Translated Text:</h2>
        <p>{{ translated_text }}</p>
    {% endif %}

    <h1>Why make an Arabic to English Translator</h1>
    <p>bc we wanted to lmao</p>
</body>
</html>
"""

@app.route('/')
def index():
    return render_template_string(template)

@app.route('/translate', methods=['POST'])
def translate():
    text = request.form['text']
    res = pipeline.fullAnnotate(text)

    translated_text = " ".join([sentence.result for sentence in res[0]['translation']])
    return render_template_string(template, original_text=text, translated_text=translated_text)

# Run Flask app
from pyngrok import ngrok

# Set  ngrok authtoken
NGROK_AUTH_TOKEN = "YOUR AUTH TOKEN"
#this token is from my uni account
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

public_url = ngrok.connect(5000)
print(f'Public URL: {public_url}')
app.run(port=5000)
